In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import eyecite
from datetime import datetime
from markdown import markdown
from IPython.display import Markdown, display, HTML
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from typing import Tuple, Optional
from src.schema.decisions import Decision
from src.schema.decisions import extract_parallel_citation_context, filter_decisions_by_forward_citations
from src.doc_store.cap_client import CAPClient
from src.doc_store.cl_client  import CourtListenerClient
from src.utils.citations import create_highlighted_citation
from src.doc_store.base import process_citations_with_progress
from src.doc_store.utils import (
    load_models_from_json, 
    save_models_to_json, 
    looks_like_case_law_link, 
    filter_decisions_with_text,
)

In [3]:
top_citations = pd.read_excel("./data/tables/top_citations.xlsx")
top_citations["cap_ind"] = top_citations["link"].apply(looks_like_case_law_link)
Markdown(top_citations.to_markdown())

|    | Citation              |   Count | link                                                      | cap_ind   |
|---:|:----------------------|--------:|:----------------------------------------------------------|:----------|
|  0 | 120 Cal. Rptr. 2d 162 |     226 | https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162 | False     |
|  1 | 908 A.2d 888          |     197 | https://cite.case.law/a2d/908/888                         | True      |
|  2 | 750 P.2d 297          |     195 | https://cite.case.law/p2d/750/297                         | True      |
|  3 | 121 F.3d 496          |     190 | https://cite.case.law/f3d/121/496                         | True      |
|  4 | 268 S.W.3d 487        |     187 | https://cite.case.law/sw3d/268/487                        | True      |
|  5 | 693 P.2d 796          |     186 | https://cite.case.law/p2d/693/796                         | True      |
|  6 | 36 Cal. App. 4th 500  |     184 | https://cite.case.law/cal-app-4th/36/500                  | True      |
|  7 | 40 P.2d 311           |     182 | https://cite.case.law/p2d/40/311                          | True      |
|  8 | 159 F. Supp. 2d 1196  |     178 | https://cite.case.law/f-supp-2d/159/1196                  | True      |
|  9 | 44 Cal. Rptr. 2d 370  |     177 | https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370  | False     |

In [3]:
cl_client = CourtListenerClient()
cap_client = CAPClient()

In [4]:

def fetch_case(citation: str) -> Tuple[Optional[Decision], bool]:
    """
    Fetches a case by citation, trying CAPClient first and falling back to CourtListenerClient if necessary.

    Args:
        citation (str): The citation string to search for.

    Returns:
        Tuple[Optional[Decision], bool]: A tuple containing the Decision model (or None if not found) and a boolean indicating if the case was found in CAP (True) or CourtListener (False).
    """
    try:
        case = cap_client.read_cite(citation, full_case=True, body_format="html")
        return case, True
    except (ValueError, IndexError):
        try:
            cite = cl_client.find_cite(citation)
            case = cl_client.fetch_case_by_id(cite.id)
            return case, False
        except:
            return None, False

def update_case_counts(case: Decision, is_cap: bool) -> Decision:
    """
    Updates the Decision model with counts of "cites to" and "cited by" cases.

    Args:
        case (Decision): The Decision model to update.
        is_cap (bool): Indicates if the case was found in CAP (True) or CourtListener (False).

    Returns:
        Decision: The updated Decision model.
    """
    if is_cap:
        forward_cites = cap_client.get_forward_citation_ids(case.id)
    else:
        forward_cites = cl_client.get_forward_citation_ids(case.id)
    
    case_updated = case.model_copy(update={"forward_citation_ids": forward_cites})
    cites_to_count = len(case_updated.cites_to)
    cited_by_count = len(case_updated.forward_citation_ids)
    
    return case_updated, cites_to_count, cited_by_count


def process_citations(top_citations: pd.DataFrame) -> Tuple[pd.DataFrame, list]:
    """
    Processes citations in a DataFrame, updating each with "cites to" and "cited by" counts, and collects updated Decision objects.

    Args:
        top_citations (pd.DataFrame): The DataFrame containing citations to process.

    Returns:
        Tuple[pd.DataFrame, list]: The updated DataFrame with new counts and a list of updated Decision objects.
    """
    updated_decisions = []
    
    for idx, row in top_citations.iterrows():
        citation = row["Citation"]
        case, is_cap = fetch_case(citation)
        
        if case:
            case_updated, cites_to_count, cited_by_count = update_case_counts(case, is_cap)
            print(f"'{str(case_updated)}' cites to {cites_to_count} cases & been cited {cited_by_count} times.")
            
            updated_decisions.append(case_updated)
        else:
            print(f"Error reading: {citation}")
            cites_to_count = cited_by_count = 0
        
        top_citations.at[idx, 'cites_to_count'] = cites_to_count
        top_citations.at[idx, 'cited_by_count'] = cited_by_count
    
    return top_citations, updated_decisions


In [6]:
top_citations_updated, decisions_list = process_citations(top_citations)
print(len(decisions_list))

'Cunningham, 120 Cal. Rptr. 2d 162 (2002-04-30)' cites to 22 cases & been cited 7 times.
'Kvaerner Metals Division of Kvaerner U.S., Inc. v. Commercial Union Insurance, 589 Pa. 317 (2006-10-25)' cites to 37 cases & been cited 100 times.
'Isaacson, 750 P.2d 297 (1988-03-07)' cites to 0 cases & been cited 0 times.
'National Steel Corp. v. Golden Eagle Insurance, 121 F.3d 496 (1997-08-06)' cites to 49 cases & been cited 41 times.
'Zurich American Insurance Co. v. Nokia, Inc., 268 S.W.3d 487 (2008-08-29)' cites to 54 cases & been cited 97 times.
'Brandt, 693 P.2d 796 (1985-01-28)' cites to 1 cases & been cited 20 times.
'Pruyn v. Agricultural Insurance, 36 Cal. App. 4th 500 (1995-06-30)' cites to 93 cases & been cited 62 times.
Error reading: 40 P.2d 311
'Everett Associates, Inc. v. Transcontinental Insurance, 159 F. Supp. 2d 1196 (2001-08-28)' cites to 117 cases & been cited 7 times.
'Waller, 900 P.2d 619 (1995-08-28)' cites to 14 cases & been cited 20 times.
9


In [7]:
Markdown(top_citations_updated.to_markdown())

|    | Citation              |   Count | link                                                      | cap_ind   |   cites_to_count |   cited_by_count |
|---:|:----------------------|--------:|:----------------------------------------------------------|:----------|-----------------:|-----------------:|
|  0 | 120 Cal. Rptr. 2d 162 |     226 | https://www.courtlistener.com/c/Cal.%20Rptr.%202d/120/162 | False     |               22 |                7 |
|  1 | 908 A.2d 888          |     197 | https://cite.case.law/a2d/908/888                         | True      |               37 |              100 |
|  2 | 750 P.2d 297          |     195 | https://cite.case.law/p2d/750/297                         | True      |                0 |                0 |
|  3 | 121 F.3d 496          |     190 | https://cite.case.law/f3d/121/496                         | True      |               49 |               41 |
|  4 | 268 S.W.3d 487        |     187 | https://cite.case.law/sw3d/268/487                        | True      |               54 |               97 |
|  5 | 693 P.2d 796          |     186 | https://cite.case.law/p2d/693/796                         | True      |                1 |               20 |
|  6 | 36 Cal. App. 4th 500  |     184 | https://cite.case.law/cal-app-4th/36/500                  | True      |               93 |               62 |
|  7 | 40 P.2d 311           |     182 | https://cite.case.law/p2d/40/311                          | True      |                0 |                0 |
|  8 | 159 F. Supp. 2d 1196  |     178 | https://cite.case.law/f-supp-2d/159/1196                  | True      |              117 |                7 |
|  9 | 44 Cal. Rptr. 2d 370  |     177 | https://www.courtlistener.com/c/Cal.%20Rptr.%202d/44/370  | False     |               14 |               20 |

In [8]:
cl_cites = top_citations_updated[top_citations_updated["cap_ind"]==False]["Citation"].tolist()
cap_cites = [
    "121 F.3d 496",
    "268 S.W.3d 487",
]
len(cl_cites), len(cap_cites)

(2, 2)

In [9]:
cl_cases = []

for citation in cl_cites:
    cite = cl_client.find_cite(citation)
    forward_decisions = cl_client.read_forward_citations(cite.id)
    cl_cases.extend(forward_decisions)
    
print(len(cl_cases))    

Fetching forward citation ID: 2441985
Citation: Btr Hampstead, LLC v. Source Interlink, 5 A.3d 142, 194 Md. App. 538 (Md. Ct. Spec. App. 2010)
Fetching forward citation ID: 2439427
Citation: Brucia v. Hartford Acc. and Indem., 307 F. Supp. 2d 1079 (N.D. Cal. 2003)
Fetching forward citation ID: 2251305
Citation: Davis v. Farmers Ins. Group, 134 Cal. App. 4th 100, 35 Cal. Rptr. 3d 738 (Cal. Ct. App. 2005)
Fetching forward citation ID: 2740011
Citation: Baek v. Continental Casualty Co.,  (Cal. Ct. App. 2014)
Fetching forward citation ID: 2473426
Citation: Lakeland Village Owners Association v. Great American Insurance Company, 727 F. Supp. 2d 887 (E.D. Cal. 2010)
Fetching forward citation ID: 2276439
Citation: Mercury Ins. Co. v. Ayala, 11 Cal. Rptr. 3d 158, 116 Cal. App. 4th 1198 (Cal. Ct. App. 2004)
Fetching forward citation ID: 2273431
Citation: BAROCO WEST, INC. v. Scottsdale Ins. Co., 110 Cal. App. 4th 96, 1 Cal. Rptr. 3d 464 (Cal. Ct. App. 2003)
Fetching forward citation ID: 1462146

In [10]:
cite = cap_client.read_cite(cap_cites[0])

In [11]:
read_id = cap_client.read_id(cite.id)

In [12]:
read_id.text_clean

'NATIONAL STEEL CORPORATION, a Delaware corporation, Plaintiff-Appellee, v. GOLDEN EAGLE INSURANCE COMPANY, Defendant-Appellant. No. 95-17291. United States Court of Appeals, Ninth Circuit. Argued and Submitted July 17, 1997. Decided Aug. 6, 1997. *497 Irving H. Greines and Marc J. Poster, Greines, Martin, Stein, & Richland, Beverly Hills, CA, for defendant-appellant. *498 John L. Condrey, Gordon & Rees, San Francisco, CA, for plaintiff-appellee. Before: FARRIS and TASHIMA, Circuit Judges, and STAGG, Senior District Judge. * * The Honorable Tom Slagg, Senior United States District Judge for the Western District of Louisiana, sitting by designation. FARRIS, Circuit Judge: Golden Eagle appeals a district court summary judgment order holding Golden Eagle liable for the stipulated settlement reached by Golden Eagle’s insured after Golden Eagle denied defense to the insured because the claim did not fall with the insured’s policy. The district court had jurisdiction pursuant to 28 U.S.C. § 

In [13]:
cap_cases = []

for citation in cap_cites:
    try:
        cite = cap_client.read_cite(citation)
        forward_decisions = cap_client.read_forward_citations(cite.id)
        cap_cases.extend(forward_decisions)
    except IndexError as e:
        print(f"Error: {e}")
        pass
    
print(len(cap_cases))

Fetching forward citation ID: 8971904


Citation: Bassidji v. Goe, 413 F.3d 928 (2005-06-15)
Fetching forward citation ID: 5388163
Citation: Atmel Corp. v. St. Paul Fire & Marine, 426 F. Supp. 2d 1039 (2005-10-11)
Fetching forward citation ID: 5713928
Citation: Smith v. Pacific Bell Telephone Co., 649 F. Supp. 2d 1073 (2009-08-12)
Fetching forward citation ID: 9499657
Citation: Yates v. Rexton, Inc., 267 F.3d 793 (2001-10-05)
Fetching forward citation ID: 3787146
Citation: Smith v. Pacific Bell Telephone Co., 662 F. Supp. 2d 1199 (2009-08-12)
Fetching forward citation ID: 4127501
Citation: Carlson v. Century Surety Co., 832 F. Supp. 2d 1086 (2011-05-27)
Fetching forward citation ID: 9169166
Citation: Church Mutual Insurance v. United States Liability Insurance, 347 F. Supp. 2d 880 (2004-04-26)
Fetching forward citation ID: 8448529
Citation: Boyd v. City of Oakland, 458 F. Supp. 2d 1015 (2006-10-17)
Fetching forward citation ID: 5632914
Citation: Waimea Bay Associates One, LLC v. Young, 438 F. Supp. 2d 1186 (2006-06-05)
Fetch

In [14]:
from itertools import chain

case_list = list(chain(decisions_list, cl_cases, cap_cases))
len(case_list)

174

In [15]:
decisions_list[1].model_dump()

{'decision_date': '2006-10-25',
 'name': 'KVAERNER METALS DIVISION OF KVAERNER U.S., INC. f/k/a Kvaerner Davy Division of Kvaerner U.S. Inc. f/k/a Davy International, A Division of Trafalgar House, Inc. and Kvaerner Songer, Inc., f/k/a Kvaerner Davy Songer, Inc. f/k/a Davy Songer, Inc., Together A Joint Venture; Kvaerner Metals Division of Kvaerner U.S., Inc. f/k/a Kvaerner Davy Division of Kvaerner U.S. Inc f/k/a Davy International, A Division of Trafalgar House, Inc., Kvaerner Songer, Inc., f/k/a Kvaerner Davy Songer, Inc. f/k/a Davy Songer, Inc.; Kvaerner Public Liability Company f/k/a Trafalgar House Public Liability Company; and Kvaerner ASA v. COMMERCIAL UNION INSURANCE COMPANY, Lexington Insurance Company, National Union Fire Insurance Company of Pittsburgh, PA, Bethlehem Steel Corporation, Thyssen Still Otto Anlagentechnick GMBH, Successor-in-interest to Still Otto, GMBH. Appeal of National Union Fire Insurance Company Of Pittsburgh, PA.',
 'name_abbreviation': 'Kvaerner Metals

In [16]:
top_four = cl_cites + cap_cites
top_four_df = pd.DataFrame({"Citation": top_four})
top_four_updated, decisions = process_citations(top_four_df)
Markdown(top_four_updated.to_markdown())

'Cunningham, 120 Cal. Rptr. 2d 162 (2002-04-30)' cites to 22 cases & been cited 7 times.
'Waller, 900 P.2d 619 (1995-08-28)' cites to 14 cases & been cited 20 times.
'National Steel Corp. v. Golden Eagle Insurance, 121 F.3d 496 (1997-08-06)' cites to 49 cases & been cited 41 times.
'Zurich American Insurance Co. v. Nokia, Inc., 268 S.W.3d 487 (2008-08-29)' cites to 54 cases & been cited 97 times.


|    | Citation              |   cites_to_count |   cited_by_count |
|---:|:----------------------|-----------------:|-----------------:|
|  0 | 120 Cal. Rptr. 2d 162 |               22 |                7 |
|  1 | 44 Cal. Rptr. 2d 370  |               14 |               20 |
|  2 | 121 F.3d 496          |               49 |               41 |
|  3 | 268 S.W.3d 487        |               54 |               97 |

In [17]:
case_list = decisions + cl_cases + cap_cases
len(case_list)

169

In [4]:
# save_models_to_json(case_list, "./data/models/top_four.json")
cases = load_models_from_json(Decision, "./data/models/top_four.json")

In [5]:
forward_cases = filter_decisions_by_forward_citations(cases[0], cases)
len(forward_cases)

7

In [6]:
citations = [citation.cite for citation in cases[0].citations]
citations

['120 Cal. Rptr. 2d 162', '98 Cal. App. 4th 1141']

In [7]:
for i in range(len(forward_cases)):
    forward_cases[i].set_citation_context(citations)
    display(Markdown(forward_cases[i].context))

This in no way interfered with Fidelitone's use of the premises. (Citations omitted) (emphasis added). In its initial brief in this Court, BTR argues that its actions did not constitute a constructive eviction. This argument, however, invokes the wrong doctrine of law, because the doctrine of constructive eviction is not applicable to the facts of the case sub judice. [2] In its reply brief, BTR clarifies that the circuit court erred in finding that BTR evicted Source Interlink from the property. According to BTR, Source Interlink (1) had decommissioned the property and was only using it to store equipment, (2) "continued to store and showcase property in the facility" even after Fidelitone had left, (3) was informed at all times by BTR that Fidelitone's occupancy was only temporary, and (4) "maintained the keys to the premises and continued to access the property weekly." Such facts, argues BTR, demonstrate that BTR did not "interfere[ ] with any of the activities which Source [Interlink] conducted in the space prior to, during or after the flood." Because Source Interlink's "use of the space and its activities with respect to *152 the space never changed," BTR argues that it was "impossible to have an actual eviction." We disagree and explain. Our review of Maryland case law has disclosed no definition of "actual eviction." Black's Law Dictionary 35 (6th ed.1990) defines actual eviction as "[a]n actual expulsion of the tenant out of all or some part of the demised premises. A physical ouster or dispossession from the very thing granted or some substantial part thereof. " (Emphasis added). Thompson on Real Property § 41.03(c)(2), entitled "Actual Eviction by the Landlord," explains: The landlord will be in breach of the implied covenant of quiet enjoyment if the landlord, or someone for whose acts the landlord is responsible, wrongfully ousts the tenant from the leased premises. The cases refer to this conduct, which can take the form of physical expulsion or a physical exclusion of the tenant from the leased premises, as an actual eviction. Particular acts by the landlord that have been found to be sufficient to constitute actual eviction of a tenant have included changing the locks on the leased premises, refusing to permit the tenant to enter a building in which space has been leased, padlocking the entry to the leased premises, and taking possession of the leased premises with threats of violence against the tenant or members of the tenant's family if they attempt to re-enter. The cases also recognize that the landlord's eviction efforts may deprive the tenant of only a portion of the leased premises, leaving the tenant in possession of the remainder. If the landlord deprives the tenant of a significant portion of the leased premises, the landlord's conduct will generally be regarded as constituting a partial actual eviction, and a breach of the implied covenant. (Emphasis added) (footnotes omitted); see also Cunningham v. Universal Underwriters, 98 Cal. App. 4th 1141 , 120 Cal. Rptr. 2d 162 , 168-69 (2002) ("The legal definition of an eviction is the same: an `eviction' is `[t]he act or process of legally dispossessing a person of land or rental property.' (Black's Law Dict. (7th ed.1999) p. 575, col. 2.) A wrongful eviction thus occurs when the person recovering the property had no right to dispossess the other party from the property." (Alteration in original)); cf. Day v. Watson, 8 Mich. 535 , 536 (1860) ("[T]he entry being followed by a continuous possession, which was inconsistent with the possessory title assured to the tenants under the lease, that possession amounts very clearly to an eviction."). The Restatement (Second) of Property: Landlord and Tenant § 6.1 (hereinafter "the Restatement"), provides in pertinent part: Except to the extent the parties to a lease validly agree otherwise, there is a breach of the landlord's obligations if, during the period the tenant is entitled to possession of the leased property, the landlord, or someone whose conduct is attributable to him, interferes with a permissible use of the leased property by the tenant. (Emphasis added). Comment (a) to section 6.1 of the Restatement states that "[t]he conduct in § 6.1 is more commonly referred to as an eviction by the landlord." (Emphasis added). Comment (b) to section 6.1 of the Restatement further explains that "[a]n unauthorized possession of all or any part of the leased property by the landlord, or someone whose conduct is attributable to him, is an eviction of the tenant that could be cured by the tenant himself." One of the Illustrations given in Comment (b) is particularly relevant to the instant case: *153 L leases land to T. T enters the leased property. Six months later L, without T's consent, takes possession of a building located on a remote corner of the leased premises that T is not currently using. L's possession of part of the leased property is unauthorized, and L is interfering with a permissible use thereof by T. The actions of BTR in allowing Fidelitone to take possession of the premises leased to Source Interlink are more extensive than the Illustration discussed above. Source Interlink leased approximately 126,786 square feet of the Hampstead Facility, of which 114,891 square feet was warehouse space and 11,895 square feet was office space. Clark, a member of BTR, "testified that Fidelitone used and occupied 115,000 square feet of Source Interlink's space, constituting substantially all of Source Interlink's warehouse space." Although there was testimony that Source Interlink stored "a large piece of equipment" in the warehouse during the time Fidelitone occupied the premises, the impact of such equipment on the warehouse space used by Fidelitone was negligible. The equipment referred to was Source Interlink's air system, consisting mainly of a Twinn City Fan and Blower, which was mounted on the roof. Source Interlink wanted to sell the air system, and estimated that its removal would cost $8,500.00.

Here, R & H, tenants of the Property, have sued Plaintiffs, their landlords, for invading their right of private occupancy to the Property, not merely based on the contamination of the property, but also based on Plaintiffs' alleged failure to inspect and disclose contamination. Accordingly, the Court finds that the Underlying Complaint alleges personal injury claims separate from claims for property damage. [1] 3) Applicability of the Pollution Exclusion The Hartford argues that, even if the Court finds that the Underlying Complaint alleges personal injury damages, the Second Pollution Exclusion bars coverage. The Court disagrees. Although the Second Pollution Exclusion does not expressly state that it applies only to property damage or bodily injury, both pollution exclusions are listed as exclusions under Section IA of the insurance contract, the section of *1085 the contract dealing with coverage for bodily injury and property damage liability. Coverage for personal injury claims, on the other hand, is found in Section IB, a separate section containing its own insuring agreement and applicable exclusions. Given the structure of the insurance contract, an insured would reasonably expect that the exclusions listed under Section IA of the insurance contract, including both pollution exclusions, apply only to property damage or bodily injury. See Legarra v. Federated Mut. Ins. Co., 35 Cal. App.4th 1472 , 1477, 42 Cal.Rptr.2d 101 (1995)(finding that, given the format of the policy, an insured would reasonably expect the pollution exclusion to apply only to coverage for bodily injury and property damage, despite the lack of specific reference in the pollution exclusion to bodily injury and property damage). Therefore, the Court finds that the pollution exclusions in the insurance contract do not bar coverage for personal injury claims against the insured, and The Hartford has a duty to defend Plaintiffs in the suit filed by R & H. III. CONCLUSION For the foregoing reasons, the Court GRANTS Plaintiffs' motion for partial summary judgment and DENIES The Hartford's Motion to Dismiss. IT IS SO ORDERED. NOTES [1] Plaintiffs have also argued that the allegations in the Underlying Complaint cannot be claims for "property damage," since the property damage section of the policy applies only to "tangible property." Plaintiffs argue that a leasehold interest is not "tangible property" and therefore damage to a leasehold interest is not property damage as defined in the policy. No California court has directly considered whether a tenant in possession of a property has a tangible property right. In Kazi v. State Farm Fire and Casualty Co., 24 Cal.4th 871 , 103 Cal.Rptr.2d 1 , 15 P.3d 223 (2001), the Supreme Court of California held that an easement represents only a nonpossessory right to use another's property, and therefore is not tangible property. Id. at 881, 103 Cal.Rptr.2d 1 , 15 P.3d 223 . In Cunningham v. Universal Underwriters, 98 Cal. App.4th 1141 , 120 Cal.Rptr.2d 162 (2002), a California appellate court applied the reasoning of Kazi to hold that a nonpossessing tenant, whose claimed loss concerned solely the interference with its right to possession of real property under the parties' lease, did not allege damage to a tangible property right. As the court in Cunningham explained, "[a] tenant's right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession of the premises merely gives the tenant a right to abandon the tenancy and sue for damages, but does not give the tenant the legal right to act as a possessor of the real property until the date of actual transfer." Id. at 1155-1156, 120 Cal.Rptr.2d 162 . The clear implication of the Cunningham court's reasoning is that when a tenant legally takes possession of the premises, the tenant, unlike an easement holder or a nonpossessing tenant, has a possessory property right in the premises. As such, a tenant in possession presumably has a tangible property interest in the premises and can assert claims for property damage, as defined in the insurance contract.

The Davises filed a motion for summary adjudication asking the trial court to conclude that Fire owed a duty to defend the underlying action and that it breached that duty. Fire moved for summary judgment or in the alternative, summary adjudication, arguing that all causes of action against it lacked merit because there was no potential for coverage under either policy. The trial court denied the Davises' motion, granted Fire's summary judgment motion and entered judgment in favor of Fire. The Davises timely appealed. DISCUSSION I. Standard of Review We review the trial court's decision granting summary judgment de novo. ( Aguilar v. Atlantic Richfield Co. (2001) 25 Cal.4th 826 , 860, 107 Cal.Rptr.2d 841 , 24 P.3d 493 ( Aguilar ).) After identifying the issues framed by the pleadings, we determine whether the moving party has established *742 facts justifying judgment in its favor. If the moving party has carried its initial burden, we then decide whether the opposing party has demonstrated the existence of a triable, material fact issue. ( Bono v. Clark (2002) 103 Cal.App.4th 1409 , 1432, 128 Cal.Rptr.2d 31 .) We must strictly construe the moving party's evidence and liberally construe the opposing party's evidence ( Binder v. Aetna Life Ins. Co. (1999) 75 Cal. App.4th 832 , 838-839, 89 Cal.Rptr.2d 540 ) and we may not weigh the evidence or conflicting inferences. ( Aguilar, supra, 25 Cal.4th at p. 856, 107 Cal.Rptr.2d 841 , 24 P.3d 493 ; Code Civ. Proc., § 437c, subd. (c).) II. Applicable Insurance Law Principles Interpretation of an insurance policy presents a question of law governed by the general rules of contract interpretation. ( Waller v. Truck Ins. Exchange, Inc. (1995) 11 Cal.4th 1 , 18, 44 Cal.Rptr.2d 370 , 900 P.2d 619 ( Waller ).) We must give effect to the intent of the parties when they formed the contract and, if possible, infer this intent solely from the written provisions of the contract. ( E.M.M.I., Inc. v. Zurich American Ins. Co. (2004) 32 Cal.4th 465 , 470, 9 Cal. Rptr.3d 701 , 84 P.3d 385 .) In so doing, we must "look first to the language of the [insurance policy] in order to ascertain its plain meaning or the meaning a layperson would ordinarily attach to it." ( Waller, supra, 11 Cal.4th at p. 18, 44 Cal.Rptr.2d 370 , 900 P.2d 619 .) The language of the policy must be read in the context of the instrument as a whole under the circumstances of the case and a policy provision will be considered ambiguous when it is capable of two or more reasonable constructions. ( Id. at pp. 18-19, 44 Cal.Rptr.2d 370 , 900 P.2d 619 .) If a policy provision is ambiguous, we resolve the ambiguity in the insureds' favor, consistent with the insureds' reasonable expectations. ( Cunningham v. Universal Underwriters (2002) 98 Cal.App.4th 1141 , 1149, 120 Cal.Rptr.2d 162 .) An insurer must defend any action that seeks damages potentially within the coverage of the policy; however, no duty to defend exists when the third party complaint "`can by no conceivable theory raise a single issue [that] could bring it within the policy coverage.'" ( Montrose Chemical Corp. v. Superior Court (1993) 6 Cal.4th 287 , 299-300, 24 Cal.Rptr.2d 467 , 861 P.2d 1153 , quoting Gray v. Zurich Insurance Co. (1966) 65 Cal.2d 263 , 276, 54 Cal. Rptr. 104 , 419 P.2d 168 , fn. 15 ( Gray ), italics omitted.) Whether a third party action asserts a potentially covered claim under the policy triggering the duty to defend requires us to interpret the language of the insuring agreement and presents a question of law. ( Alex Robertson Co. v. Imperial Casualty & Indemnity Co. (1992) 8 Cal. App.4th 338 , 342-343, 10 Cal.Rptr.2d 165 ; Union Oil Co. v. International Ins. Co. (1995) 37 Cal. App.4th 930 , 936, 44 Cal.Rptr.2d 4 .) An insurer's duty to defend turns on the facts alleged in the underlying complaint, or available to the insurer from extrinsic sources at the time the claim is made. ( Montrose Chemical Corp. v. Superior Court, supra, 6 Cal.4th at p. 295, 24 Cal.Rptr.2d 467 , 861 P.2d 1153 ; Gray, supra, 65 Cal.2d at p. 276, 54 Cal.Rptr. 104 , 419 P.2d 168 .) The insured has the burden of showing that the claim fails within the scope of coverage and the insurer has the burden of proving that an otherwise covered claim is barred by a policy exclusion. ( Travelers Casualty & Surety Co. v. Superior Court (1998) 63 Cal. App.4th 1440 , 1453, 75 Cal.Rptr.2d 54 .) "[I]n an action [in which] none of the claims is even potentially covered, the insurer does not have a duty to defend." ( Buss v. Superior Court (1997) 16 Cal.4th 35 , 47, 65 Cal.Rptr.2d 366 , 939 P.2d 766 .) *743 III. Analysis A. Policy Coverage Provisions The first step in any insurance coverage dispute is to determine whether the insuring provisions of the policy afforded coverage for the alleged losses. The policies at issue are typical homeowner's policies, divided into two separate and distinct parts: (1) first party coverage whereby the insurer agrees to indemnify its insured for specific losses to the designated property; and (2) personal liability insurance to the named insureds against third party claims against them. (See Garvey v. State Farm Fire & Casualty Co. (1989) 48 Cal.3d 395 , 399 & fn. 2, 257 Cal.Rptr. 292 , 770 P.2d 704 .) This action does not involve first party coverage; rather, it involves the personal liability portion of the policies, which provides liability coverage to the insureds, limited only by the specific exclusionary provisions of each policy. Because the personal liability portion of each policy is substantially the same, our coverage discussion applies to both policies.

Instead, they represented a “stepping away” from HMWC’s business because they were performed solely for Baek’s own benefit, not for HMWC’s. They thus cannot be said to have occurred “while performing duties related to the conduct of [HMWC’s] business.” For the same reasons, we conclude that the alleged sexual assault was not “with respect to the conduct of [HMWC’s] business.” Although HMWC’s business created the opportunity for the alleged assault, the assault was not done at HMWC’s request or for its benefit. To paraphrase Lisa M., a deliberate sexual assault “is fairly attributed not to any peculiar aspect of [a massage enterprise], but only to ‘propinquity and lust.’” (Lisa M., 14 supra, 12 Cal.4th at p. 302.) It therefore cannot be characterized as having occurred “with respect to” the conduct of HMWC’s business. ”2 Baek contends that the phrase “but only with respect to the conduct of [HMWC’s] business” should be read broadly, to connote only a minimal causal connection or incidental relationship. Neither of the cases Baek cites in support of this argument concern sexual misconduct, however, and thus neither stands for the proposition that sexual harassment or sexual assault has the requisite causal connection to Baek’s employment to give rise to coverage under an employer’s CGL policy. (Hartford Casualty Ins. Co. v. Travelers Indemnity Co. (2003) 110 Cal. App.4th 710 , 714 [held: commercial tenant’s CGL insurer owed landlord a duty of defense in premises liability action brought by tenant’s employee’s parents; rejecting insurer’s contention that “‘but only with respect to’” refers only to liability directly caused by landlord]; Acceptance Ins. Co. v. Syufy Enterprises (1999) 69 Cal. App.4th 321 , 328 [building owner was additional insured under contractor’s CGL policy when contractor’s employee was injured while performing repairs on building’s roof; “[employee’s] injury clearly ‘arose out of’ the work he was performing on the roof of Syufy’s building. The relationship between the defective [roof] hatch and the job was more than incidental, in that [employee] could not have done the job without passing through the hatch.” ].) D. Baek’s Additional Contentions Are Without Merit 1. Negligence Baek contends that even if there was no duty to defend with regard to the allegations of sexual assault, there was a duty to defend Jaime W.’s claim of negligence. We do not agree. “The scope of the duty does not depend on the labels given to the causes of action in the third party complaint; instead it rests on whether the alleged facts . . . reveal a possibility that the claim may be covered by the policy.” (Atlantic Mutual Ins. Co. v. J. Lamb, Inc. (2002) 100 Cal.App.4th 1017 , 1034, italics omitted; 2 Having so concluded, we do not address Continental’s additional contention that it did not owe Baek a duty of defense because he did not timely tender his claim. 15 Cunningham v. Universal Underwriters (2002) 98 Cal.App.4th 1141 , 1148 [same].) Here, although Jaime W. asserts a cause of action for negligent massage, the sole facts alleged in support describe intentional conduct—i.e., that Baek “touched, fondled, rubbed, grabbed and squeezed Plaintiff’s breasts, buttocks, inner thighs and genitals, all while making and emitting moans, groans, grunts and other sounds and noises of sexual pleasure.” These alleged acts of sexual misconduct “are, by their very nature, intentional” (Coit Drapery Cleaners, Inc. v. Sequoia Ins. Co. (1993) 14 Cal. App.4th 1595 , 1606)—to paraphrase Shanahan v. State Farm General Ins. Co. (2011) 193 Cal. App.4th 780 , 788, one does not “accidentally” sexually fondle another. Thus, the cause of action for negligence did not give rise to a duty to defend. Jaime W.’s complaint also alleges that each defendant negligently hired, trained, and supervised Baek. Although this allegation on its face is alleged against Baek, as well as the other defendants, we conclude that Baek cannot be liable for failing to use due care while supervising himself. (See State Farm Fire & Casualty Co. v. Century Indemnity Co. (1997) 59 Cal. App.4th 648 , 663 [“We next note that State Farm treats Derrick and the other teachers the same with respect to the claims based on the failure to report. However, in doing so, State Farm conveniently ignores obvious and material differences between them. The other teachers’ potential liability arose solely from their failure to properly respond to information about sexual misconduct by another teacher. . . . [¶] [I]f [insurer] had a duty to defend Derrick against claims based on failure to report, what would emerge is a simple way to plead around the caselaw that precludes a duty to defend teachers against claims of molestation. One need only allege that the teacher/molester negligently failed to report the information known to him or her that would lead a reasonable person to suspect a student had been molested.” ].) 2. False Imprisonment Baek contends Continental had a duty to defend because Jaime W.’s complaint contained a cause of action for false imprisonment, for which there is coverage under the Continental policy. Again, we do not agree. Courts have found no duty to defend 16 allegations of nonsexual misconduct that are “inseparably intertwined” with allegations of intentional sexual misconduct. For example, in Jane D. v. Ordinary Mutual (1995) 32 Cal.App.4th 643 , 653, the plaintiff’s allegations of nonsexual conduct—“obtaining information about plaintiff during counseling and using this information and misusing counseling techniques to create transference and to control and induce plaintiff’s behavior”—were “‘inseparably intertwined’” with the allegations of sexual assault. The court explained: “Plaintiff’s complaint alleges [defendant’s] misuse of his counseling position resulted in the inducement for her to engage in sexual relations with him. He used the information he learned about plaintiff to influence and control her behavior and he utilized the feelings created by transference to create a sexual relationship. None of the allegations of [defendant’s] malfeasance in counseling stands separate from the allegation of sexual misconduct.

(Pl.'s Evid., Ex. 12, at 1.) "Personal injury" is defined to include "injury, other than `bodily injury,' arising out of ... the wrongful eviction from, wrongful entry into, or invasion of the right of private occupancy of a room, dwelling or premises that a person occupies by or on behalf of its owner, landlord or lessor." (Id., at 3.) III. DISCUSSION The parties dispute whether or not Defendant owes Plaintiff a defense of Hollow's Cross-Complaint. Plaintiff seeks a declaration that Defendant has a duty to defend it, arguing Hollow's nuisance claim is covered by the policy's coverage for injury arising out of "invasion of the right of private occupancy of a room, dwelling or premises that a person occupies by or on behalf of its owner, landlord or lessor...." (Pl.'s P. & A. in Supp. of Mot. for Partial Summ. J. ("Mot.") 8:9-19.) Defendant rejoins that it does not owe Plaintiff a defense because "invasion of the right of private occupancy," requires a physical invasion, and "[n]oise simply is not a physical invasion of the right to private occupancy in California." (Def.'s P. & A. in Opp'n to Pl.'s Mot. for Summ. J. ("Opp'n") 9:8-10, 10:20-21.) Defendant further counters that it does not owe Plaintiff a defense since the clause "by or on behalf of its owner, landlord or lessor," modifies the word "invasion," requiring Plaintiff to own the affected property for there to be coverage, and Plaintiff does not own Hollow's property. (Opp'n 7:11-15.) A. An Insurer's Duty to Defend Under California law, An insurer must defend its insured against claims that create a potential for indemnity under the policy. The duty to defend is broader than the duty to indemnify, and it may apply even in an action where no damages are ultimately awarded. Determination of the duty to defend depends, in the first instance, on a comparison between the allegations of the complaint and the terms of the policy. But the duty also exists where extrinsic facts known to the insurer suggest that the claim may be covered. Moreover, that the precise causes of action pled by the third-party complaint may fall outside policy coverage does not excuse the duty to defend where, under the facts alleged, reasonably inferable, or otherwise known, the complaint could fairly be amended to state a covered liability. Scottsdale Ins. Co. v. MV Transp., 36 Cal. 4th 643 , 654, 31 Cal. Rptr. 3d 147 , 115 P.3d 460 (2005) (citation omitted). Further, "[a]ny doubt as to whether the facts give rise to a duty to defend is resolved in the insured's favor." Horace Mann Ins. Co. v. Barbara B., 4 Cal. 4th 1076 , 1081, 17 Cal. Rptr. 2d 210 , 846 P.2d 792 (1993). In a duty to defend case, an insured moving for summary judgment "need only show `the existence of a potential for coverage,' i.e., `that the underlying claim may fall within policy coverage.'" Cunningham v. Univ. Underwriters, 98 Cal. App. 4th 1141 , 1147, 120 Cal. Rptr. 2d 162 (2002) (citing Montrose Chem. Corp. v. Sup. Ct., 6 Cal. 4th 287 , 300, 24 Cal. Rptr. 2d 467 , 861 P.2d 1153 (1993)). Whereas, an insurer moving for summary judgment "`must establish the absence of any ... potential' for coverage, i.e., that the underlying complaint `can by no conceivable theory raise a single issue which could bring *891 it within the policy coverage.'" Id. (citing Montrose Chem. Corp., at 300, 24 Cal. Rptr. 2d 467 , 861 P.2d 1153 (1993)). "Although the insurer's burden in moving for summary judgment is greater than the insured's burden in bringing its own affirmative motion, this disparity merely reflects the substantive law." Id. (citation omitted). When an insurer owes a duty of defense, "the insurer is obligated to defend against all of the claims involved in the action, both covered and noncovered...." Horace Mann Ins. Co. v. Barbara B., 4 Cal. 4th 1076 , 1081, 17 Cal. Rptr. 2d 210 , 846 P.2d 792 (1993). "To defend meaningfully, the insurer must defend immediately. To defend immediately, it must defend entirely. It cannot parse the claims, dividing those that are at least potentially covered from those that are not." Buss v. Sup. Ct., 16 Cal. 4th 35 , 49, 65 Cal. Rptr. 2d 366 , 939 P.2d 766 (1997). B. Applicable Principals of Insurance Policy Interpretation The principles of insurance policy interpretation are well-settled under California law: Interpretation of an insurance policy is a question of law. While insurance contracts have special features, they are still contracts to which ordinary rules of contractual interpretation apply. Thus, the mutual intention of the parties at the time the contract is formed governs interpretation. If possible, [courts] infer this intent solely from the written provisions of the insurance policy. If the policy language is clear and explicit, it governs. When interpreting a policy provision, [courts] must give its terms their ordinary and popular sense unless used by the parties in a technical sense or a special meaning is given to them by usage. We must also interpret these terms in context, and give effect to every part of the policy with each clause helping to interpret the other. Palmer v. Truck Ins. Exchange, 21 Cal. 4th 1109 , 1115, 90 Cal. Rptr. 2d 647 , 988 P.2d 568 (1999) (quotations and citations omitted). "A policy provision will be considered ambiguous when it is capable of two or more constructions, both of which are reasonable. But language in a contract must be interpreted as a whole, and in the circumstances of the case, and cannot be found to be ambiguous in the abstract." Waller v. Truck Ins. Exchange, Inc., 11 Cal. 4th 1 , 18, 44 Cal. Rptr. 2d 370 , 900 P.2d 619 (1995). "There cannot be an ambiguity per se, i.e. an ambiguity unrelated to an application." Bay Cities Paving & Grading, Inc. v. Lawyers' Mutual Ins. Co., 5 Cal. 4th 854 , 867, 21 Cal. Rptr.

We affirm. Factual and Procedural Summary In October of 1991, Medina was struck by a car driven by an uninsured motorist. She worked for Robinsons-May, and the accident took place in the employee parking *160 lot. Robinsons-May, which is self-insured for workers' compensation claims, paid her medical and other expenses in an amount which exceeded $15,000. Medina and Ayala [1] had automobile insurance with Mercury. The declarations page lists uninsured motorist bodily injury coverage limits of $15,000 per person and $30,000 per accident. Ayala and Medina made claims under that coverage. Mercury denied the claims, then sued for declaratory relief, seeking a declaration that both claims were subject to a single $15,000 per person limit and that the limit was exhausted because it was reduced by the workers' compensation benefits paid to Medina. Appellants cross-complained for declaratory relief, breach of contract, and bad faith. Mercury moved for summary judgment on the complaint. The court granted the motion, finding that the claims were not covered. Mercury demurred to the amended cross-complaint on that ground. The demurrer was sustained and judgment was entered in Mercury's favor. Discussion 1. Mercury's reliance on a specimen policy [**] 2. Coverage: Which limit applies? [2] Part IV of appellants' policy is entitled "Uninsured Motorists Coverage." In it, Mercury promises to pay "all sums which the insured ... shall be legally entitled to recover as damages from the owner, or operator of an uninsured motor vehicle because of bodily injury, sustained by the insured, caused by accident and arising out of the ... use of such uninsured motor vehicles...." The next part of the policy, titled "Part IV  Conditions," includes the provision that "The limit of liability stated in the declarations as applicable to `each person' is the limit of the company's liability for all damages arising out of bodily injury sustained by one person in any one accident, ... For the purposes of this provision, the `bodily injury sustained by any one person' as used herein, shall be deemed to include all injury and damages for care, loss of consortium and injury to any interpersonal relationship sustained by others as a consequence of such bodily injury." The second quoted sentence is in bold face. At summary judgment, Mercury contended that the unambiguous language of this clause meant that the per person limit of $15,000, not the per accident limit of $30,000, applies to appellants' claims. Appellants make two claims, that the per accident limit applies under the terms of the policy and that the per accident limit must apply, because Insurance Code section 11580.2 mandates that it does. As to the policy itself, we agree with Mercury that the language is unambiguous. We thus give effect to its plain meaning ( Cunningham v. Universal Underwriters (2002) 98 Cal. App. 4th 1141 , 1149, 120 Cal. Rptr. 2d 162 ), which is clearly the one Mercury ascribes to it  the per person limit applies. Numerous cases beginning with United Services Automobile Assn. v. Warner (1976) 64 Cal. App. 3d 957 , 135 Cal. Rptr. 34 have construed similar language in liability policies and have found that the *161 language is unambiguous and means what Mercury, and we, say that it means. Appellants seek a different result, citing Abellon v. Hartford Ins. Co. (1985) 167 Cal. App. 3d 21 , 212 Cal. Rptr. 852 . Abellon, like Warner, is a declaratory relief action filed by an insurer after its insured was in a car accident and was sued not only by the other driver, but by that driver's spouse on a loss of consortium claim. Neither case involves uninsured motorist coverage, but in both cases, the insurer sought a declaration that the per person, not the per accident, limit of the liability coverage applied. Warner found that the per person limit applied, and Abellon found that the per accident limit applied. However, in Warner, the policy language was like the language before us, [3] and in Abellon there was no similar language defining "bodily injury to any one person" to include the loss of consortium sustained by another. Instead, the insured in Abellon case had "no notice when it purchased the policy that loss of consortium damages fell within the purview of the `per person' limitation." ( Id. at p. 31, 212 Cal. Rptr. 852 .) Abellon does not help us construe appellants' policy. We next consider appellants' statutory argument. Insurance Code section 11580.2 sets out the minimum uninsured motorist coverage which every car insurance policy must include. It is designed to ensure that drivers injured by uninsured motorists drivers are protected to the same extent they would have been if the driver at fault had carried the statutory minimum of liability insurance. If a policy conflicts with the statute, the statute prevails. ( Hartford Casualty Ins. Co. v. Cancilla (1994) 28 Cal. App. 4th 1305 , 1310, 34 Cal. Rptr. 2d 302 .) Insurance Code section 11580.2 sets out the minimum coverage in two ways. First, it provides that unless the insured agrees in writing to the contrary, the uninsured motorist coverage must have limits at least equal to the limits of liability for bodily injury in the underlying policy of insurance. (Ins.Code, § 11580.2, subd. (a)(1); (m).) Appellants' policy complies with this requirement. The uninsured motorist bodily injury coverage limits are the same as the bodily injury limits in the rest of the policy. Indeed, the definition of "bodily injury sustained by any one person" is the same in both coverages.

*472 4. Disposition We affirm the trial court's judgment. Scottsdale shall recover its costs on appeal. We concur: RAMIREZ, P.J., and WARD, J. NOTES [1] Guz v. Bechtel National, Inc. (2000) 24 Cal.4th 317 , 334, 100 Cal.Rptr.2d 352 , 8 P.3d 1089 . [2] Parsons v. Crown Disposal Co. (1997) 15 Cal.4th 456 , 500, 63 Cal.Rptr.2d 291 , 936 P.2d 70 . [3] See Code of Civil Procedure section 437c, subdivision (c); Guz v. Bechtel National, Inc., supra, 24 Cal.4th at page 334, 100 Cal.Rptr.2d 352 , 8 P.3d 1089 . [4] See Smith Kandal Real Estate v. Continental Casualty Co. (1998) 67 Cal. App.4th 406 , 414, 79 Cal.Rptr.2d 52 . [5] Smith Kandal Real Estate v. Continental Casualty Co., supra, 67 Cal. App.4th at pages 413-414, 79 Cal.Rptr.2d 52 . [6] Smith Kandal Real Estate v. Continental Casualty Co., supra, 67 Cal. App.4th at page 414, 79 Cal.Rptr.2d 52 . [7] See Waller v. Truck Ins. Exchange, Inc. (1995) 11 Cal.4th 1 , 18, 44 Cal.Rptr.2d 370 , 900 P.2d 619 . [8] See Waller v. Truck Ins. Exchange, Inc., supra, 11 Cal.4th at page 18, 44 Cal.Rptr.2d 370 , 900 P.2d 619 ; Buxbaum v. Aetna Life & Casualty Co. (2002) 103 Cal. App.4th 434 , 442, 126 Cal.Rptr.2d 682 . [9] See Waller v. Truck Ins. Exchange, Inc., supra, 11 Cal.4th at page 19, 44 Cal.Rptr.2d 370 , 900 P.2d 619 . [10] Waller v. Truck Ins. Exchange, Inc., supra, 11 Cal.4th at page 19, 44 Cal.Rptr.2d 370 , 900 P.2d 619 . [11] American Internat. Bank v. Fidelity & Deposit Co. (1996) 49 Cal. App.4th 1558 , 1571, 57 Cal.Rptr.2d 567 . [12] See Montrose Chemical Corp. v. Superior Court (1993) 6 Cal.4th 287 , 297-298, 24 Cal. Rptr.2d 467 , 861 P.2d 1153 ; Swain v. California Casualty Ins. Co. (2002) 99 Cal. App.4th 1 , 8, 120 Cal.Rptr.2d 808 ; Northland Ins. Co. v. Briones (2000) 81 Cal. App.4th 796 , 806, 97 Cal.Rptr.2d 127 ; Hurley Construction Co. v. State Farm Fire & Casualty Co. (1992) 10 Cal. App.4th 533 , 538, 12 Cal.Rptr.2d 629 . [13] See Buena Vista Mines, Inc. v. Industrial Indemnity Co. (2001) 87 Cal. App.4th 482 , 487, 104 Cal.Rptr.2d 557 . [14] See FMC Corp. v. Plaisted & Companies (1998) 61 Cal. App.4th 1132 , 1154-1155, 72 Cal.Rptr.2d 467 . [15] FMC Corp. v. Plaisted & Companies, supra, 61 Cal. App.4th at page 1155, 72 Cal.Rptr.2d 467 ; see also Ringler Associates Inc. v. Maryland Casualty Co. (2000) 80 Cal. App.4th 1165 , 96 Cal.Rptr.2d 136 . [16] Aerojet-General Corp. v. Transport Indemnity Co. (1997) 17 Cal.4th 38 , 59, 70 Cal. Rptr.2d 118 , 948 P.2d 909 . [17] Aerojet-General Corp. v. Transport Indemnity Co., supra, 17 Cal.4th at page 59, 70 Cal.Rptr.2d 118 , 948 P.2d 909 . [18] See Montrose Chemical Corp. v. Superior Court, supra, 6 Cal.4th at page 300, 24 Cal. Rptr.2d 467 , 861 P.2d 1153 ; Cunningham v. Universal Underwriters (2002) 98 Cal.App.4th 1141 , 1147, 120 Cal.Rptr.2d 162 . [19] Hurley Construction Co. v. State Farm Fire & Casualty Co., supra, 10 Cal. App.4th at page 538, 12 Cal.Rptr.2d 629 . [20] Cutler-Orosi Unified School Dist. v. Tulare County School etc. Authority (1994) 31 Cal. App.4th 617 , 633, 37 Cal.Rptr.2d 106 . [21] See Nissel v. Certain Underwriters at Lloyd's of London (1998) 62 Cal.App.4th 1103 , 1111, 73 Cal.Rptr.2d 174 . [22] But see Roger H. Proulx & Co. v. Crest-Liners, Inc. (2002) 98 Cal. App.4th 182 , 203, 119 Cal.Rptr.2d 442 (involving facts indicating possible damage to property other than tank liner on which contractor was performing work). [23] See Buena Vista Mines, Inc. v. Industrial Indemnity Co., supra, 87 Cal. App.4th at page 488, 104 Cal.Rptr.2d 557 , quoting Waller v. Truck Ins. Exchange, Inc.. supra, 11 Cal.4th at page 36, 44 Cal.Rptr.2d 370 , 900 P.2d 619 ; San Diego Housing Com. v. Industrial Indemnity Co. (1998) 68 Cal.

In [8]:
decisions_context = extract_parallel_citation_context(forward_cases, citations)
len(decisions_context)

7

In [9]:
decisions_context[4].parallel_context

'(Pl.\'s Evid., Ex. 12, at 1.) "Personal injury" is defined to include "injury, other than `bodily injury,\' arising out of ... the wrongful eviction from, wrongful entry into, or invasion of the right of private occupancy of a room, dwelling or premises that a person occupies by or on behalf of its owner, landlord or lessor." (Id., at 3.) III. DISCUSSION The parties dispute whether or not Defendant owes Plaintiff a defense of Hollow\'s Cross-Complaint. Plaintiff seeks a declaration that Defendant has a duty to defend it, arguing Hollow\'s nuisance claim is covered by the policy\'s coverage for injury arising out of "invasion of the right of private occupancy of a room, dwelling or premises that a person occupies by or on behalf of its owner, landlord or lessor...." (Pl.\'s P. & A. in Supp. of Mot. for Partial Summ. J. ("Mot.") 8:9-19.) Defendant rejoins that it does not owe Plaintiff a defense because "invasion of the right of private occupancy," requires a physical invasion, and "[n]

In [10]:
Markdown(create_highlighted_citation(decisions_context[1].parallel_context, str(citations[1])))

Here, R & H, tenants of the Property, have sued Plaintiffs, their landlords, for invading their right of private occupancy to the Property, not merely based on the contamination of the property, but also based on Plaintiffs' alleged failure to inspect and disclose contamination. Accordingly, the Court finds that the Underlying Complaint alleges personal injury claims separate from claims for property damage. [1] 3) Applicability of the Pollution Exclusion The Hartford argues that, even if the Court finds that the Underlying Complaint alleges personal injury damages, the Second Pollution Exclusion bars coverage. The Court disagrees. Although the Second Pollution Exclusion does not expressly state that it applies only to property damage or bodily injury, both pollution exclusions are listed as exclusions under Section IA of the insurance contract, the section of *1085 the contract dealing with coverage for bodily injury and property damage liability. Coverage for personal injury claims, on the other hand, is found in Section IB, a separate section containing its own insuring agreement and applicable exclusions. Given the structure of the insurance contract, an insured would reasonably expect that the exclusions listed under Section IA of the insurance contract, including both pollution exclusions, apply only to property damage or bodily injury. See Legarra v. Federated Mut. Ins. Co., 35 Cal. App.4th 1472 , 1477, 42 Cal.Rptr.2d 101 (1995)(finding that, given the format of the policy, an insured would reasonably expect the pollution exclusion to apply only to coverage for bodily injury and property damage, despite the lack of specific reference in the pollution exclusion to bodily injury and property damage). Therefore, the Court finds that the pollution exclusions in the insurance contract do not bar coverage for personal injury claims against the insured, and The Hartford has a duty to defend Plaintiffs in the suit filed by R & H. III. CONCLUSION For the foregoing reasons, the Court GRANTS Plaintiffs' motion for partial summary judgment and DENIES The Hartford's Motion to Dismiss. IT IS SO ORDERED. NOTES [1] Plaintiffs have also argued that the allegations in the Underlying Complaint cannot be claims for "property damage," since the property damage section of the policy applies only to "tangible property." Plaintiffs argue that a leasehold interest is not "tangible property" and therefore damage to a leasehold interest is not property damage as defined in the policy. No California court has directly considered whether a tenant in possession of a property has a tangible property right. In Kazi v. State Farm Fire and Casualty Co., 24 Cal.4th 871 , 103 Cal.Rptr.2d 1 , 15 P.3d 223 (2001), the Supreme Court of California held that an easement represents only a nonpossessory right to use another's property, and therefore is not tangible property. Id. at 881, 103 Cal.Rptr.2d 1 , 15 P.3d 223 . In Cunningham v. Universal Underwriters, <mark>98 Cal. App.4th 1141</mark> , 120 Cal.Rptr.2d 162 (2002), a California appellate court applied the reasoning of Kazi to hold that a nonpossessing tenant, whose claimed loss concerned solely the interference with its right to possession of real property under the parties' lease, did not allege damage to a tangible property right. As the court in Cunningham explained, "[a] tenant's right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession of the premises merely gives the tenant a right to abandon the tenancy and sue for damages, but does not give the tenant the legal right to act as a possessor of the real property until the date of actual transfer." Id. at 1155-1156, 120 Cal.Rptr.2d 162 . The clear implication of the Cunningham court's reasoning is that when a tenant legally takes possession of the premises, the tenant, unlike an easement holder or a nonpossessing tenant, has a possessory property right in the premises. As such, a tenant in possession presumably has a tangible property interest in the premises and can assert claims for property damage, as defined in the insurance contract.

In [11]:
import nest_asyncio
nest_asyncio.apply()

all_data = await process_citations_with_progress(forward_cases)

100%|██████████| 7/7 [01:22<00:00, 11.78s/it]


In [12]:
data_dicts = [model.model_dump() for model in all_data]
df_res = pd.DataFrame(data_dicts)
Markdown(df_res.to_markdown(index=False))

| citation              | legal_question                                                                                                                                                                                                          | rule                                                                                                                                                                                                                                                                                                                                                                                                                                      | application                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | citation_reference                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|:----------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 120 Cal. Rptr. 2d 162 | Does allowing a third party to take possession of premises leased to another constitute actual eviction?                                                                                                                | A wrongful eviction occurs when the person recovering the property had no right to dispossess the other party from the property.                                                                                                                                                                                                                                                                                                          | The court found that wrongful eviction occurs under the legal definition when there is a dispossession of a person from land or rental property that they have rights to, without proper legal basis.                                                                                                                                                                                                                                                                                    | The case of Cunningham v. Universal Underwriters was cited to support the legal interpretation of eviction as the act of legally dispossessing a person of land or rental property, and further clarify that wrongful eviction occurs under circumstances involving illegal dispossession.                                                                                                                                                                                                           |
| 120 Cal. Rptr. 2d 162 | Does a tenant have a tangible property right in the premises before physically taking possession, and can they assert claims for property damage as defined in the insurance contract before actual possession?         | A tenant's right to possess property on the lease commencement date is a contractual right that does not mature into a property right until possession actually occurs. A landlord's failure to deliver possession of the premises merely gives the tenant a right to abandon the tenancy and sue for damages, but does not give the tenant the legal right to act as a possessor of the real property until the date of actual transfer. | The court in the mentioned case applied the ruling that a nonpossessing tenant, whose claimed loss concerned solely the interference with their right to possession of real property under the parties' lease, did not allege damage to a tangible property right. This reasoning implies that when a tenant legally takes possession of the premises, they then, and only then, have a possessory property right in the premises, thus potentially having a tangible property interest. | The citation was used to establish that a tenant's leasehold interest does not constitute a tangible property right until actual possession of the property, distinguishing between contractual rights and tangible property rights in the context of insurance coverage for property damage.                                                                                                                                                                                                        |
| 120 Cal. Rptr. 2d 162 | In an insurance policy dispute, how should ambiguous policy provisions be interpreted, and who benefits from such interpretations?                                                                                      | If a policy provision is ambiguous, the ambiguity should be resolved in the insureds' favor, consistent with the insureds' reasonable expectations.                                                                                                                                                                                                                                                                                       | The rule suggests that in cases where the language of an insurance policy can be interpreted in multiple reasonable ways, the interpretation that favors the insured's expectations should be chosen. This approach helps in protecting the interests of the insured party by ensuring that ambiguities do not work to their disadvantage.                                                                                                                                               | The citation is referenced to support the principle that ambiguities in insurance policy provisions must be resolved in favor of the insured, based on their reasonable expectations. This principle is applied within the broader context of interpreting and evaluating insurance policy disputes, especially in situations where the policy language is unclear or open to more than one reasonable interpretation.                                                                               |
| 98 Cal. App. 4th 1141 | Does a cause of action for negligence give rise to a duty to defend against claims of sexual assault in the context of employer liability under a commercial general liability (CGL) policy?                            | The scope of the duty to defend does not depend on the labels given to the causes of action in a third party complaint, but rather on whether the alleged facts reveal a possibility that the claim may be covered by the policy.                                                                                                                                                                                                         | The cause of action for negligence did not give rise to a duty to defend due to the intentional nature of the alleged acts of sexual misconduct, which are by their very nature, intentional and not accidental. Claims for negligent hiring, training, and supervising were also not applicable to Baek, as he could not be held liable for failing to use due care while supervising himself.                                                                                          | The citation was referenced to support the principle that the determination of a duty to defend based on a negligence claim revolves around the nature of the facts alleged rather than the labels of the causes of action. It emphasized that mere claims of negligence related to sexual misconduct, which intrinsically entail intentional acts, do not establish a duty to defend under a CGL policy.                                                                                            |
| 120 Cal. Rptr. 2d 162 | Does the insurance policy cover the plaintiff's claim for defense against a nuisance claim related to 'invasion of the right of private occupancy' where the alleged invasion was non-physical but rather due to noise? | An insurer must defend its insured against claims creating a potential for indemnity under the policy, even if the precise causes of action pled may fall outside policy coverage. Doubt as to whether the facts give rise to a duty to defend is resolved in the insured's favor. The insured must only show the existence of a potential for coverage, while an insurer must establish the absence of any potential for coverage.       | In this case, if the nuisance claim may potentially fall within the insurance policy's coverage for 'invasion of the right of private occupancy,' even if the invasion is non-physical, the insurer could have a duty to defend against the claim.                                                                                                                                                                                                                                       | The citation was used to discuss the principles determining an insurer's duty to defend its insured, emphasizing that the insured need only demonstrate a potential for coverage to argue for the insurer's duty to defend, whereas the insurer must prove no possibility of coverage to be relieved of this duty.                                                                                                                                                                                   |
| 120 Cal. Rptr. 2d 162 | Does the per person limit or the per accident limit of uninsured motorist bodily injury coverage apply to the claimants' cases?                                                                                         | The per person limit applies to the claimants' cases based on the unambiguous language of the insurance policy's clause.                                                                                                                                                                                                                                                                                                                  | The court agreed with Mercury that the language of the insurance policy is unambiguous and gives effect to its plain meaning, which is that the per person limit, not the per accident limit, applies to appellants' claims.                                                                                                                                                                                                                                                             | The court referenced Cunningham v. Universal Underwriters, stating they give effect to the plain language of the insurance policy as was similarly found in Cunningham v. Universal Underwriters (2002) 98 Cal. App. 4th 1141, 120 Cal. Rptr. 2d 162, which states that the unambiguous language of an insurance policy should be given its plain meaning.                                                                                                                                           |
| 120 Cal. Rptr. 2d 162 | What legal principles or questions were addressed in the case referenced by the citation 120 Cal. Rptr. 2d 162?                                                                                                         |                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | The citation 120 Cal. Rptr. 2d 162 was referenced in the context of discussing diverse legal principles or rulings. It is associated with Cunningham v. Universal Underwriters (2002) 98 Cal.App.4th 1141, 1147. The reference appears to be part of an extensive discussion that includes various cases and legal principles, indicating that the case may have been used to support or illustrate a specific legal argument or principle in the broader context of the legal discussion presented. |

In [13]:
from src.utils.pydantic_utils import *
from lancedb.pydantic import LanceModel, Vector

In [87]:
fields = [
    "id",
    "decision_date",
    "name",
    "judges",
    "attorneys",
    "head_matter",
    "frontend_url", 
    "data",
    "status",
    "forward_citation_ids",
    "cites_to",    
]

In [108]:
cases= filter_decisions_with_text(cases)
len(cases)

167

In [119]:
fields_dicts = []
for i in range(len(cases)):
    try:
        case_dict = extract_fields(cases[i], fields)
        if not isinstance(cases[i].casebody.data, str):
            case_dict["type"] = cases[i].casebody.data.opinions[0].type if cases[i].casebody.data.opinions else None
        else:
            case_dict["type"] = "Missing"
        case_dict["content"] = cases[i].text_clean
        case_dict["html"] = cases[i].text
        case_dict["citation"] = cases[i].citations if cases[i].citations else None
        fields_dicts.append(case_dict)
    except AttributeError as e:
        print(f"{e}")
        pass
df_cases = pd.DataFrame(fields_dicts)

In [124]:
print(f"df cases shape: {df_cases.shape}")
df_cases.sample(2)

df cases shape: (167, 16)


,id,decision_date,name,judges,attorneys,head_matter,frontend_url,status,forward_citation_ids,cites_to,type,content,html,citation,data,tokens
124,9992,2015-06-26,"MCGINNES INDUSTRIAL MAINTENANCE CORPORATION, A...",NaN,None,NaN,https://cite.case.law/sw3d/477/786/,ok,None,"[243 S.W.2d 154, 150 Tex. 513, 46 F.3d 1116, 2...",Missing,"MCGINNES INDUSTRIAL MAINTENANCE CORPORATION, A...","<section class=""casebody"" data-case-id=""320441...",[477 S.W.3d 786],"<section class=""casebody"" data-case-id=""320441...",19983
127,8820,2011-01-07,"VRV DEVELOPMENT L.P., formerly known as VRV De...",NaN,None,NaN,https://cite.case.law/f3d/630/451/,ok,None,"[100 F.2d 441, 955 F.2d 1353, 434 F.3d 165, 56...",Missing,"VRV DEVELOPMENT L.P., formerly known as VRV De...","<section class=""casebody"" data-case-id=""320441...",[630 F.3d 451],"<section class=""casebody"" data-case-id=""320441...",5427


In [121]:
from src.utils.gen_utils import count_tokens

df_cases["tokens"] = df_cases["content"].apply(lambda x: count_tokens(x))

In [122]:
df_cases["tokens"].describe()

count      167.000000
mean      8883.143713
std       6724.815578
min        640.000000
25%       4809.500000
50%       7385.000000
75%      10944.500000
max      54689.000000
Name: tokens, dtype: float64

In [123]:
df_cases.to_excel("./data/tables/cases_with_text.xlsx", index=False)

In [125]:
top_four[0]

'120 Cal. Rptr. 2d 162'

In [126]:
str(cases[0])

'Cunningham, 120 Cal. Rptr. 2d 162 (2002-04-30)'

In [127]:
cited_by_test = cl_client.read_cited_by(cases[0].id)

Fetching case ID: 1160464
Citation: Vons Companies, Inc. v. Seabest Foods, Inc., 14 Cal. 4th 434 (Cal. 1996)
Fetching case ID: 2258291
Citation: General Accident Ins. Co. v. West Am. Ins. Co., 42 Cal. App. 4th 95, 49 Cal. Rptr. 2d 603 (Cal. Ct. App. 1996)
Fetching case ID: 2302806
Citation: Martin Marietta Corp. v. Insurance Co. of N. Am., 40 Cal. App. 4th 1113, 47 Cal. Rptr. 2d 670 (Cal. Ct. App. 1995)
Fetching case ID: 2253301
Citation: Fibreboard Corp. v. Hartford Acc. & Indemnity Co., 16 Cal. App. 4th 492, 20 Cal. Rptr. 2d 376 (Cal. Ct. App. 1993)
Fetching case ID: 2293434
Citation: Hendrickson v. ZURICH AMERICAN INS. CO., 85 Cal. Rptr. 2d 622, 72 Cal. App. 4th 1084 (Cal. Ct. App. 1999)
Fetching case ID: 2144332
Citation: Bartlome v. State Farm Fire & Casualty Co., 208 Cal. App. 3d 1235, 256 Cal. Rptr. 719 (Cal. Ct. App. 1989)
Fetching case ID: 2270816
Citation: Lee v. Placer Title Co., 28 Cal. App. 4th 503, 33 Cal. Rptr. 2d 572 (Cal. Ct. App. 1994)
Fetching case ID: 1158094
Citati

In [128]:
len(cited_by_test)

22

In [129]:
cited_by_test[0].text_clean

'14 Cal.4th 434 (1996) VONS COMPANIES, INC., Cross-complainant and Appellant, \nv. \nSEABEST FOODS, INC., et al., Cross-defendants and Respondents. Docket No. S049039. Supreme Court of California. December 12, 1996. *440 COUNSEL Munger, Tolles & Olson, Gregory P. Stone, Joseph D. Lee, Kristin A. Linsley, David M. Rosenzweig, Susan R. Szabo and Kristin S. Escalante for Cross-complainant and Appellant. Neil, Dymott, Perkins, Brown & Frank, Thomas M. Dymott, Kyle A. Cruse, Crosby, Heafey, Roach & May, Peter W. Davis, James C. Martin, Jacques B. LeBoeuf, Laskero & Associates and Michael G. Roddy for Cross-defendants and Respondents. OPINION GEORGE, C.J. We consider in this case the issue whether California courts may exercise personal jurisdiction over owners of "fast food" restaurant franchises located in another state, in litigation stemming from several incidents of food poisoning that occurred in the foreign jurisdiction. We conclude that the franchisees had sufficient contacts with Ca

In [139]:
decisions[2].id, str(decisions[2])

(285037,
 'National Steel Corp. v. Golden Eagle Insurance, 121 F.3d 496 (1997-08-06)')

In [140]:
len(decisions[2].cites_to)

49

In [141]:
cited_by_test_2 = cap_client.read_cited_by(decisions[2].id)

Fetching case ID: 2282116
Fetched Decision: Clemmer v. Hartford Insurance, 22 Cal. 3d 865 (1978-12-19)
Fetching case ID: 2013849
Fetched Decision: Samson v. Transamerica Insurance, 30 Cal. 3d 220 (1981-11-23)
Fetching case ID: 1310362
Fetched Decision: Aetna Casualty & Surety Co. v. Centennial Insurance, 838 F.2d 346 (1988-02-01)
Fetching case ID: 11318566
Fetched Decision: Salve Regina College v. Russell, 499 U.S. 225 (1991-03-20)
Fetching case ID: 4372662
Fetched Decision: Xebec Development Partners, Ltd. v. National Union Fire Insurance, 12 Cal. App. 4th 501 (1993-01-13)
Fetching case ID: 6207800
Fetched Decision: Celotex Corp. v. Catrett, 477 U.S. 317 (1986-06-25)
Fetching case ID: 2263866
Fetched Decision: Kuchel v. McCormack, 104 Cal. App. 2d 506 (1951-05-31)
Fetching case ID: 4881724
Fetched Decision: Pruyn v. Agricultural Insurance, 36 Cal. App. 4th 500 (1995-06-30)
Fetching case ID: 10513471
Fetched Decision: Koch Industries, Inc. v. Mobil Oil Corp., 968 F.2d 27 (1992-04-07)
F

In [142]:
cited_by_test_2[0].text_clean

'[L.A. No. 30759. Dec. 19, 1978.] MARJORIE J. CLEMMER et al., Plaintiffs and Appellants, v. HARTFORD INSURANCE COMPANY, Defendant and Appellant. *871 Counsel Gibson, Dunn & Crutcher, John H. Sharer and Fred F. Gregory for Plaintiffs and Appellants. Overton, Lyman & Prince, Carl J. Schuck, John D. McCurdy, Valerie Baker and Gwen H. Whitson for Defendants and Appellants. Opinion MANUEL, J. By this action plaintiffs, the widow and the son of a victim of a killing, sue to recover from the liability insurer of the slayer the amount of a wrongful death judgment obtained against him. Following a jury verdict the trial court entered judgment against the defendant insurer, but it subsequently granted defendant’s motion for new trial, tendered on all issues, on the sole issue submitted to the jury—i.e., whether the death in question was caused by a willful act—and denied it on all other issues. Plaintiffs appeal from the aforesaid order granting a limited new trial. Defendant appeals from that p

In [143]:
print(len(cited_by_test))
cited_by_test.extend(cited_by_test_2)
print(len(cited_by_test))

22
47


In [144]:
save_models_to_json(cited_by_test, "./data/models/cited_to.json")